In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [7]:
start_date = '20221226'
end_date = '20230326' 

In [8]:
query = f"""

with order_logs_snapshot as (
select 
    DATE_FORMAT(DATE_TRUNC('week', CAST(date_parse(yyyymmdd, '%Y%m%d') AS DATE)), '%Y-%m-%d') time_level,
    service_obj_service_name service, 
    order_id, captain_id, customer_id, 
    captain_obj_device_id, customer_obj_device_device_id,discount
from orders.order_logs_snapshot
where
    yyyymmdd BETWEEN '{start_date}' and '{end_date}' 
    and service_obj_service_name in ('Auto', 'Link')
    and order_status = 'dropped' 
    and customer_obj_device_device_id != '00000000'
    and customer_obj_device_device_id != ''
    and customer_obj_device_device_id is not null 
),

agg_data as (
select
    time_level,
    count(distinct order_id) total_orders,
    count(distinct customer_id) total_customer
    
from order_logs_snapshot
group by 1
),

customer_device_threshold as (
select
    time_level,
    customer_id,
    count(distinct customer_obj_device_device_id) mapped_device
from order_logs_snapshot
where (customer_id is not null and customer_id <> '')
group by 1,2
having count(distinct customer_obj_device_device_id) > 4
),

fraud_cus as (
select 
    time_level, 
    count(distinct customer_id) fraud_customer,
    count(distinct order_id) fraud_customer_orders,
    count(distinct case when discount > 0 then order_id end) discounted_orders,
    sum(discount) discount
from order_logs_snapshot
where customer_id in (select distinct customer_id from customer_device_threshold)
group by 1
),

coin_burn as
(
select 
    time_level,
    subscription_debit,
    adjustment_debit as adjustments,
    
    customercashback_creditcoinlocationoffer_credit + customercashback_creditcoinrideoffer_credit + scratchcard_creditcoinscratchcard_credit 
            + customercashback_credit + locationoffer_creditcoin_credit + locationoffer_creditcoinlocationoffer_credit + rideoffer_creditcoin_credit + rideoffer_creditcoinrideoffer_credit as coin_credited,
            
    utilization_dashboardcoinscredit + utilization_giftoffer + utilization_walletrechargeoffer as coin_utilized,
    coinexpired_locationoffer_debit + coinexpired_rideoffer_debit + coinexpired_scratchcardoffer_debit + coinexpired_debit as coin_expired,
    
    customerrefund_credit as refund
    from
    (
    select
        DATE_FORMAT(DATE_TRUNC('week', CAST(date_parse(yyyymmdd, '%Y%m%d') AS DATE)), '%Y-%m-%d') time_level,
        -- city_name,
        -- customer_id,
        -- service_name,
        sum(coalesce(subscription_debit, 0)) as subscription_debit,
        sum(coalesce(adjustment_debit, 0)) as adjustment_debit,
        sum(coalesce(customercashback_creditcoinlocationoffer_credit, 0)) as customercashback_creditcoinlocationoffer_credit,
        sum(coalesce(customercashback_creditcoinrideoffer_credit, 0)) as customercashback_creditcoinrideoffer_credit,
        sum(coalesce(scratchcard_creditcoinscratchcard_credit, 0)) as scratchcard_creditcoinscratchcard_credit,
        sum(coalesce(utilization_dashboardcoinscredit, 0)) as utilization_dashboardcoinscredit,
        sum(coalesce(utilization_giftoffer, 0)) as utilization_giftoffer,
        sum(coalesce(utilization_referraloffer, 0)) as utilization_referraloffer,
        sum(coalesce(utilization_walletrechargeoffer, 0)) as utilization_walletrechargeoffer,
        sum(coalesce(coinexpired_locationoffer_debit, 0)) as coinexpired_locationoffer_debit,
        sum(coalesce(coinexpired_rideoffer_debit, 0)) as coinexpired_rideoffer_debit,
        sum(coalesce(coinexpired_scratchcardoffer_debit, 0)) as coinexpired_scratchcardoffer_debit,
        sum(coalesce(customerrefund_credit, 0)) as customerrefund_credit,
        
        sum(coalesce(coinexpired_debit,0)) as coinexpired_debit,
        sum(coalesce(customercashback_credit,0)) as customercashback_credit,
        sum(coalesce(locationoffer_creditcoin_credit,0)) as locationoffer_creditcoin_credit,
        sum(coalesce(locationoffer_creditcoinlocationoffer_credit,0)) as locationoffer_creditcoinlocationoffer_credit,
        sum(coalesce(rideoffer_creditcoin_credit,0)) as rideoffer_creditcoin_credit,
        sum(coalesce(rideoffer_creditcoinrideoffer_credit,0)) as rideoffer_creditcoinrideoffer_credit
        from datasets.customer_financial_transactions_kpi
    where 
        yyyymmdd BETWEEN '{start_date}' and '{end_date}' 
        and customer_id in (select distinct customer_id from customer_device_threshold)
        and service_name in ('Auto', 'Link')
    group by 1
    )
)

select 
    agg_data.*,
    fraud_cus.fraud_customer,
    fraud_cus.fraud_customer_orders,
    fraud_cus.discounted_orders,
    fraud_cus.discount,
    coin_burn.coin_credited,
    coin_burn.coin_utilized,
    coin_burn.coin_expired
    
from agg_data 
join fraud_cus on agg_data.time_level = fraud_cus.time_level
join coin_burn on agg_data.time_level = coin_burn.time_level
order by 1 desc

"""

In [9]:
df_code = pd.read_sql(query, conn) 

In [6]:
df_oct_dec = df_code
df_oct_dec

,time_level,total_orders,total_customer,fraud_customer,fraud_customer_orders,discounted_orders,discount,coin_credited,coin_utilized,coin_expired
0,2022-12-19,5812417,2784249,107,668,190,2423.0,36.0,0.0,0.0
1,2022-12-12,5808238,2802748,180,1082,193,2339.0,216.0,0.0,8.0
2,2022-12-05,5638506,2748267,179,1083,203,2466.0,259.0,20.0,41.0
3,2022-11-28,5738617,2736700,160,819,182,1961.0,377.0,20.0,104.0
4,2022-11-21,5600555,2633003,139,1000,252,3123.0,680.0,30.0,96.0
5,2022-11-14,5636437,2647073,175,889,220,2597.0,103.0,0.0,36.0


In [10]:
df_dec_mar = df_code
df_dec_mar

,time_level,total_orders,total_customer,fraud_customer,fraud_customer_orders,discounted_orders,discount,coin_credited,coin_utilized,coin_expired
0,2023-03-20,5331923,2636557,55,226,68,644.0,60.0,0.0,0.0
1,2023-03-13,5632327,2736823,78,400,113,1094.0,153.0,50.0,7.0
2,2023-03-06,5340797,2688015,92,452,130,1453.0,26.0,20.0,32.0
3,2023-02-27,5870934,2811664,127,643,167,1792.0,178.0,86.0,64.0
4,2023-02-20,5825493,2794369,120,569,173,1758.0,149.0,130.0,9.0
5,2023-02-13,5718383,2784105,217,1322,233,2371.0,243.0,299.0,30.0
6,2023-02-06,5956803,2867544,225,1686,303,2978.0,168.0,166.0,144.0
7,2023-01-30,5840191,2804659,211,1055,343,3391.0,250.0,175.0,24.0
8,2023-01-23,5399276,2708799,185,951,287,2611.0,293.0,2.0,14.0
9,2023-01-16,5184353,2561013,145,776,266,2814.0,65.0,114.0,0.0


In [17]:
frames = [df_oct_dec, df_dec_mar]
result = pd.concat(frames)
result.to_csv("cus 1.csv", index = False)

In [18]:
result

,time_level,total_orders,total_customer,fraud_customer,fraud_customer_orders,discounted_orders,discount,coin_credited,coin_utilized,coin_expired
0,2022-12-19,5812417,2784249,107,668,190,2423.0,36.0,0.0,0.0
1,2022-12-12,5808238,2802748,180,1082,193,2339.0,216.0,0.0,8.0
2,2022-12-05,5638506,2748267,179,1083,203,2466.0,259.0,20.0,41.0
3,2022-11-28,5738617,2736700,160,819,182,1961.0,377.0,20.0,104.0
4,2022-11-21,5600555,2633003,139,1000,252,3123.0,680.0,30.0,96.0
5,2022-11-14,5636437,2647073,175,889,220,2597.0,103.0,0.0,36.0
0,2023-03-20,5331923,2636557,55,226,68,644.0,60.0,0.0,0.0
1,2023-03-13,5632327,2736823,78,400,113,1094.0,153.0,50.0,7.0
2,2023-03-06,5340797,2688015,92,452,130,1453.0,26.0,20.0,32.0
3,2023-02-27,5870934,2811664,127,643,167,1792.0,178.0,86.0,64.0
